# DeepFM Tensorflow Horovod on SageMaker Sample

### In this sample, we will demo how to run a deepfm sample code in tensorflow horovod on sagemaker

Notice:

1. Dataset format is TFRecord

2. This model training we will use **GPU** instances

3. Using [SageMaker Python SDK 2.x](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)

In [1]:
import sagemaker
print(sagemaker.__version__)

2.25.1


## File mode

In [18]:
#下面用多个spot实例进行parameter server方式的分布式训练。
import sagemaker
from sagemaker.tensorflow.estimator import TensorFlow
from datetime import datetime
import os

bucket = 'sagemaker-us-west-2-169088282855'
checkpoint_s3_uri = 's3://{}/deepfm-checkpoint'.format(bucket) #Change to your own path if you want to save ckpt during training
checkpoint_local_path = '/opt/ml/checkpoints'
model_dir = '/opt/ml/model'
output_path= 's3://{}/deepfm-2021'.format(bucket)

training_channel_name = 'training'
evaluation_channel_name = 'evaluation'

train_instance_type = 'ml.p3.8xlarge'
hvd_processes_per_host = 4
train_instance_count= 1

train_use_spot_instances = True
enable_s3_shard = True

train_max_run=36000*2
train_max_wait = 72000 if train_use_spot_instances else None

distributions = {'mpi': {
                    'enabled': True,
                    'processes_per_host': hvd_processes_per_host,
                    'custom_mpi_options': '-verbose --NCCL_DEBUG=INFO -x OMPI_MCA_btl_vader_single_copy_mechanism=none'
                        }
                }

deep_layer = '128,64,32'

batch_size = 1024
feature_size = 117581

base_job_name='tf-scriptmode-deepfm'

hyperparameters = {'servable_model_dir': '/opt/ml/model', 'training_data_dir': '/opt/ml/input/data/training/',
                   'val_data_dir': '/opt/ml/input/data/evaluation/', 'log_steps': 10, 'num_epochs': 10, 
                   'field_size': 39, 'feature_size': feature_size, 'deep_layers': deep_layer,
                   'perform_shuffle': 0, 'batch_size': batch_size, 'pipe_mode': 0, 'enable_s3_shard': enable_s3_shard,
                   'training_channel_name': training_channel_name, 'evaluation_channel_name': evaluation_channel_name
                  }

estimator = TensorFlow(
                       #source_dir='./',
                       entry_point='DeepFM-hvd-tfrecord-vectorized-map.py',
                       model_dir=model_dir,
                       #checkpoint_s3_uri = checkpoint_s3_uri,
                       #checkpoint_local_path = checkpoint_local_path,
                       output_path= output_path,
                       instance_type=train_instance_type,
                       instance_count=train_instance_count,
                       #volume_size = 500,
                       hyperparameters=hyperparameters,
                       role=sagemaker.get_execution_role(),
                       base_job_name=base_job_name,
                       framework_version='1.15.2',
                       py_version='py3',
                       script_mode=True,
                       #input_mode='Pipe',
                       distribution=distributions,
                       use_spot_instances=train_use_spot_instances,
                       max_wait=train_max_wait,
                       max_run=train_max_run,
                       debugger_hook_config =False,
                       disable_profiler=True
                       )

In [19]:
#下面这个测试file mode
from sagemaker.inputs import TrainingInput

train_s3_uri = 's3://sagemaker-us-west-2-169088282855/tf-SM-deepctr-deepfm-sample/data-tfrecord/training/'
validate_s3_uri = 's3://sagemaker-us-west-2-169088282855/tf-SM-deepctr-deepfm-sample/data-tfrecord/val/'

if enable_s3_shard:
    train_input = TrainingInput(train_s3_uri, distribution='ShardedByS3Key')
    val_input = TrainingInput(validate_s3_uri)
else :
    train_input = TrainingInput(train_s3_uri)
    val_input = TrainingInput(validate_s3_uri)

inputs = {training_channel_name : train_input, evaluation_channel_name : val_input}

estimator.fit(inputs)

2021-02-22 11:13:18 Starting - Starting the training job...
2021-02-22 11:13:20 Starting - Launching requested ML instances......
2021-02-22 11:14:31 Starting - Preparing the instances for training......
2021-02-22 11:15:42 Downloading - Downloading input data...
2021-02-22 11:15:54 Training - Downloading the training image......
2021-02-22 11:17:17 Uploading - Uploading generated training modelWARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.

2021-02-22 11:17:06,030 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2021-02-22 11:17:06,470 sagemaker-containers INFO     Starting MPI run as worker node.
2021-02-22 11:17:06,471 sagemaker-containers INFO     Creating SSH daemon.
2021-02-22 11:17:06,478 sagemaker-containers INFO     Waiting for MPI workers to establish their SSH connections
2021-02-22 11:17:06,47

[1,3]<stderr>:INFO:tensorflow:Running local_init_op.
[1,3]<stderr>:I0222 11:17:14.429491 139627143640896 session_manager.py:500] Running local_init_op.
[1,1]<stderr>:INFO:tensorflow:Running local_init_op.
[1,1]<stderr>:I0222 11:17:14.433466 140294389569344 session_manager.py:500] Running local_init_op.
[1,3]<stderr>:INFO:tensorflow:Done running local_init_op.
[1,3]<stderr>:I0222 11:17:14.454287 139627143640896 session_manager.py:502] Done running local_init_op.
[1,1]<stderr>:INFO:tensorflow:Done running local_init_op.
[1,1]<stderr>:I0222 11:17:14.457450 140294389569344 session_manager.py:502] Done running local_init_op.
[1,2]<stderr>:INFO:tensorflow:Running local_init_op.
[1,2]<stderr>:I0222 11:17:14.492963 139842694027072 session_manager.py:500] Running local_init_op.
[1,0]<stderr>:INFO:tensorflow:Running local_init_op.
[1,0]<stderr>:I0222 11:17:14.498416 139830347523904 session_manager.py:500] Running local_init_op.
[1,2]<stderr>:INFO:tensorflow:Done running local_init_op.
[1,2]<stde


2021-02-22 11:17:44 Failed - Training job failed


UnexpectedStatusException: Error for Training job tf-scriptmode-deepfm-2021-02-22-11-13-17-852: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "mpirun --host algo-1:4 -np 4 --allow-run-as-root --display-map --tag-output -mca btl_tcp_if_include eth0 -mca oob_tcp_if_include eth0 -mca plm_rsh_no_tree_spawn 1 -bind-to socket -map-by slot -mca pml ob1 -mca btl ^openib -mca orte_abort_on_non_zero_status 1 -x NCCL_MIN_NRINGS=4 -x NCCL_SOCKET_IFNAME=eth0 -x NCCL_DEBUG=INFO -x LD_LIBRARY_PATH -x PATH -x LD_PRELOAD=/usr/local/lib/python3.6/dist-packages/gethostname.cpython-36m-x86_64-linux-gnu.so -verbose -x OMPI_MCA_btl_vader_single_copy_mechanism=none -x SM_HOSTS -x SM_NETWORK_INTERFACE_NAME -x SM_HPS -x SM_USER_ENTRY_POINT -x SM_FRAMEWORK_PARAMS -x SM_RESOURCE_CONFIG -x SM_INPUT_DATA_CONFIG -x SM_OUTPUT_DATA_DIR -x SM_CHANNELS -x SM_CURRENT_HOST -x SM_MODULE_NAME -x SM_LOG_LEVEL -x SM_FRAMEWORK_MODULE -x SM_INPUT_DIR -x SM_INPUT_CONFIG_DIR -x SM_OUTPUT_DIR -x SM_NUM_CPUS -x S

## Pipe mode

In [ ]:
#下面用多个spot实例进行parameter server方式的分布式训练。
import sagemaker
from sagemaker.tensorflow.estimator import TensorFlow
from datetime import datetime
import os

bucket = 'sagemaker-us-west-2-169088282855'
checkpoint_s3_uri = 's3://{}/deepfm-checkpoint'.format(bucket) #Change to your own path if you want to save ckpt during training
checkpoint_local_path = '/opt/ml/checkpoints'
model_dir = '/opt/ml/model'
output_path= 's3://{}/deepfm-2021'.format(bucket)

training_channel_name = 'training'
evaluation_channel_name = 'evaluation'

train_instance_type = 'ml.p3.8xlarge'
hvd_processes_per_host = 4
train_instance_count= 2

train_use_spot_instances = True
enable_s3_shard = True

train_max_run=36000*2
train_max_wait = 72000 if train_use_spot_instances else None

distributions = {'mpi': {
                    'enabled': True,
                    'processes_per_host': hvd_processes_per_host,
                    'custom_mpi_options': '-verbose --NCCL_DEBUG=INFO -x OMPI_MCA_btl_vader_single_copy_mechanism=none'
                        }
                }

deep_layer = '128,64,32'

batch_size = 1024
feature_size = 117581

base_job_name='tf-scriptmode-deepfm'

hyperparameters = {'servable_model_dir': '/opt/ml/model', 'training_data_dir': '/opt/ml/input/data/training/',
                   'val_data_dir': '/opt/ml/input/data/evaluation/', 'log_steps': 10, 'num_epochs': 10, 
                   'field_size': 39, 'feature_size': feature_size, 'deep_layers': deep_layer,
                   'perform_shuffle': 0, 'batch_size': batch_size, 'pipe_mode': 1, 'enable_s3_shard': enable_s3_shard,
                   'training_channel_name': training_channel_name, 'evaluation_channel_name': evaluation_channel_name
                  }

estimator = TensorFlow(
                       #source_dir='./',
                       entry_point='DeepFM-hvd-tfrecord-vectorized-map.py',
                       model_dir=model_dir,
                       #checkpoint_s3_uri = checkpoint_s3_uri,
                       #checkpoint_local_path = checkpoint_local_path,
                       output_path= output_path,
                       instance_type=train_instance_type,
                       instance_count=train_instance_count,
                       #volume_size = 500,
                       hyperparameters=hyperparameters,
                       role=sagemaker.get_execution_role(),
                       base_job_name=base_job_name,
                       framework_version='1.14',
                       py_version='py3',
                       script_mode=True,
                       input_mode='Pipe',
                       distribution=distributions,
                       use_spot_instances=train_use_spot_instances,
                       max_wait=train_max_wait,
                       max_run=train_max_run,
                       debugger_hook_config =False,
                       disable_profiler=True
                       )

In [ ]:
#下面这个测试pipe mode
from sagemaker.inputs import TrainingInput

train_s3_uri = 's3://sagemaker-us-west-2-169088282855/tf-SM-deepctr-deepfm-sample/data-tfrecord/training/'
validate_s3_uri = 's3://sagemaker-us-west-2-169088282855/tf-SM-deepctr-deepfm-sample/data-tfrecord/val/'

if enable_s3_shard:
    train_input = TrainingInput(train_s3_uri, distribution='ShardedByS3Key')
    val_input = TrainingInput(validate_s3_uri)
else :
    train_input = TrainingInput(train_s3_uri)
    val_input = TrainingInput(validate_s3_uri)

inputs = {'training':train_s3, 'training-2':train_s3, 'training-3':train_s3, 'evaluation': validate_s3}

inputs = {training_channel_name : train_input, evaluation_channel_name : val_input}

estimator.fit(inputs)